## Hyperparameter optimization with Random Search using Talos 

Ref: https://autonomio.github.io/docs_talos/

In [1]:
import keras
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,Flatten,Dense,Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.activations import softmax, sigmoid, relu, elu
from keras.losses import categorical_crossentropy, binary_crossentropy
import talos as ta
from talos.model.normalizers import lr_normalizer
import numpy as np
import pandas as pd
from utils import *

Using TensorFlow backend.


In [2]:
x_train = read_train_images()
x_val = read_validation_images()

In [3]:
y_train = read_train_labels()
y_val = read_validation_labels()

In [9]:
inChannel = x_train.shape[3]
x, y = x_train.shape[1], x_train.shape[2]
num_classes = len(y_train[0])

In [15]:
# specify parameters and distributions to sample from
p={'batch_size': [16, 32, 64],
   'epochs': [25, 50],
   'dropout': [0.1, 0.2],
   'optimizer': ['Adam', 'RMSprop'],
   'loss': ['categorical_crossentropy', 'binary_crossentropy'],
   'activation':['relu', 'elu'],
   'last_activation': ['sigmoid','softmax']}  

In [16]:
def get_model(x_train, y_train, x_val, y_val, params):
    """
    Creates, compiles and trains model
    :return: Trained model and model's history object
    """
    input_img = Input(shape=(x, y, inChannel))
    conv1 = Conv2D(32, (3, 3), activation=params['activation'], padding='same')(input_img) 
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) 
    conv2 = Conv2D(64, (3, 3), activation=params['activation'], padding='same')(pool1) 
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) 
    conv3 = Conv2D(128, (3, 3), activation=params['activation'], padding='same')(pool2) 

    drop1 = Dropout(rate=params['dropout'])(conv3)
    flat = Flatten()(drop1)
    den = Dense(128, activation=params['activation'])(flat)
    last = Dense(num_classes, activation=params['last_activation'])(den)
    
    full_model = Model(input_img, last)

    full_model.compile(optimizer=params['optimizer'],
                      loss=params['loss'],
                      metrics=['accuracy'])

    out = full_model.fit(x_train, y_train,
                         batch_size=params['batch_size'],
                         epochs=params['epochs'],
                         verbose=1,
                         validation_data=(x_val, y_val))

    return out, full_model

Random search is the recommended optimization strategy in Talos. This is invoked through the 'grid_downsample' argument in Scan() with a floating point value in the experiment options. For example, to randomly pick 10% of the permutations, a grid_downsample value of 0.1 is used.

In [20]:
h = ta.Scan(x_train, y_train, 
            params=p,
            model=get_model,
            dataset_name='rand_search',
            experiment_no='1',
            grid_downsample=0.05)






  0%|          | 0/21 [00:00<?, ?it/s]

Train on 326 samples, validate on 141 samples
Epoch 1/20
326/326 [==============================] - 43s 130ms/step - loss: 9.1862 - acc: 0.4018 - val_loss: 8.4591 - val_acc: 0.4752
Epoch 2/20
326/326 [==============================] - 35s 108ms/step - loss: 9.4929 - acc: 0.4110 - val_loss: 8.4591 - val_acc: 0.4752
Epoch 3/20
326/326 [==============================] - 33s 100ms/step - loss: 9.4929 - acc: 0.4110 - val_loss: 8.4591 - val_acc: 0.4752
Epoch 4/20
326/326 [==============================] - 30s 91ms/step - loss: 9.4929 - acc: 0.4110 - val_loss: 8.4591 - val_acc: 0.4752
Epoch 5/20
326/326 [==============================] - 30s 91ms/step - loss: 9.4929 - acc: 0.4110 - val_loss: 8.4591 - val_acc: 0.4752
Epoch 6/20
326/326 [==============================] - 29s 90ms/step - loss: 9.4929 - acc: 0.4110 - val_loss: 8.4591 - val_acc: 0.4752
Epoch 7/20
326/326 [==============================] - 30s 91ms/step - loss: 9.4929 - acc: 0.4110 - val_loss: 8.4591 - val_acc: 0.4752
Epoch 8/20
32






  5%|▍         | 1/21 [11:27<3:49:07, 687.37s/it]

Train on 326 samples, validate on 141 samples
Epoch 1/100
326/326 [==============================] - 33s 103ms/step - loss: 0.7609 - acc: 0.5966 - val_loss: 0.8756 - val_acc: 0.6135
Epoch 2/100
326/326 [==============================] - 35s 108ms/step - loss: 0.5799 - acc: 0.7316 - val_loss: 0.4737 - val_acc: 0.8262
Epoch 3/100
326/326 [==============================] - 37s 112ms/step - loss: 0.3476 - acc: 0.8666 - val_loss: 0.3214 - val_acc: 0.8582
Epoch 4/100
326/326 [==============================] - 34s 103ms/step - loss: 0.2735 - acc: 0.8896 - val_loss: 0.3931 - val_acc: 0.8759
Epoch 5/100
326/326 [==============================] - 35s 107ms/step - loss: 0.3713 - acc: 0.8635 - val_loss: 0.3724 - val_acc: 0.8440
Epoch 6/100
326/326 [==============================] - 33s 101ms/step - loss: 0.2271 - acc: 0.9463 - val_loss: 0.3205 - val_acc: 0.9078
Epoch 7/100
326/326 [==============================] - 31s 96ms/step - loss: 0.1373 - acc: 0.9540 - val_loss: 0.3604 - val_acc: 0.8723
Epo

326/326 [==============================] - 49s 152ms/step - loss: 0.0032 - acc: 0.9969 - val_loss: 0.9738 - val_acc: 0.8582
Epoch 61/100
326/326 [==============================] - 46s 140ms/step - loss: 0.0031 - acc: 0.9969 - val_loss: 0.9859 - val_acc: 0.8617
Epoch 62/100
326/326 [==============================] - 43s 132ms/step - loss: 0.0030 - acc: 0.9969 - val_loss: 1.0008 - val_acc: 0.8652
Epoch 63/100
326/326 [==============================] - 43s 133ms/step - loss: 0.0028 - acc: 0.9969 - val_loss: 1.0233 - val_acc: 0.8617
Epoch 64/100
326/326 [==============================] - 50s 153ms/step - loss: 0.0027 - acc: 0.9969 - val_loss: 1.0373 - val_acc: 0.8617
Epoch 65/100
326/326 [==============================] - 48s 148ms/step - loss: 0.0025 - acc: 0.9969 - val_loss: 1.0098 - val_acc: 0.8652
Epoch 66/100
326/326 [==============================] - 48s 148ms/step - loss: 0.0023 - acc: 0.9969 - val_loss: 0.9965 - val_acc: 0.8652
Epoch 67/100
326/326 [==============================] 






 10%|▉         | 2/21 [1:15:32<8:37:37, 1634.60s/it]

Train on 326 samples, validate on 141 samples
Epoch 1/100
326/326 [==============================] - 37s 112ms/step - loss: 7.6922 - acc: 0.5015 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 2/100
326/326 [==============================] - 31s 94ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 3/100
326/326 [==============================] - 34s 104ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 4/100
326/326 [==============================] - 40s 123ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 5/100
326/326 [==============================] - 48s 146ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 6/100
326/326 [==============================] - 37s 113ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 7/100
326/326 [==============================] - 35s 107ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epo

326/326 [==============================] - 46s 142ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 61/100
326/326 [==============================] - 49s 152ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 62/100
326/326 [==============================] - 45s 137ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 63/100
326/326 [==============================] - 45s 137ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 64/100
326/326 [==============================] - 55s 169ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 65/100
326/326 [==============================] - 58s 178ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 66/100
326/326 [==============================] - 52s 160ms/step - loss: 9.4411 - acc: 0.4110 - val_loss: 8.4130 - val_acc: 0.4752
Epoch 67/100
256/326 [======================>.......] 

KeyboardInterrupt: 

In [ ]:
# accessing the results data frame
h = pd.read_csv('rand_search_1.csv', delimiter = ',')
h

In [ ]:
r = ta.Reporting('rand_search_1.csv')
